In [3]:
import cv2
import numpy as np
from tensorflow.keras.models import load_model
from PyQt5.QtCore import Qt, QTimer
from PyQt5.QtGui import QPixmap, QImage
from PyQt5.QtWidgets import QApplication, QWidget, QVBoxLayout, QLabel

In [4]:
def face_detection(face_cascade, frame, scale_factor):
    frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(frame_gray, scaleFactor=scale_factor, minNeighbors=5, minSize=(30, 30), flags=cv2.CASCADE_SCALE_IMAGE)
    return faces

def preprocess_image(face, img):
    face_rect = cv2.resize(face, (24, 24))
    face_rect = face_rect.astype('float32')
    face_rect /= 255
    return face_rect

class MainApp(QWidget):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.initUI()

    def initUI(self):
        self.setWindowTitle('Driver Drowsiness Detection')
        self.setGeometry(300, 300, 600, 400)

        self.face_cascade, self.drowsiness_model = self.load_pretrained_models()
        self.capture = cv2.VideoCapture(0)
        self.timer = QTimer()
        self.timer.timeout.connect(self.update_frame)
        self.timer.start(5)

        layout = QVBoxLayout()
        self.label = QLabel()
        self.label.setFixedSize(600, 400)
        layout.addWidget(self.label)
        self.setLayout(layout)

        self.show()

    def update_frame(self):
        ret, frame = self.capture.read()
        if ret:
            faces = face_detection(self.face_cascade, frame, 1.3)
            for (x, y, w, h) in faces:
                cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)
                face = frame[y:y+h, x:x+w]
                face_rect = preprocess_image(face, frame)
                face_pred = self.drowsiness_model.predict(np.array([face_rect]))[0]
                if face_pred > 0.5:
                    label = 'Drowsy'
                    color = (0, 0, 255)
                else:
                    label = 'Awake'
                    color = (0, 255, 0)
                cv2.putText(frame, label, (x, y), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

            image = QPixmap.fromImage(self.convert_cv_qt(frame))
            self.label.setPixmap(image)
            self.label.setAlignment(Qt.AlignCenter)

    def convert_cv_qt(self, frame):
        return QPixmap.fromImage(QImage(frame.data, frame.shape[1], frame.shape[0], QImage.Format_RGB888).rgbSwapped())

    def load_pretrained_models(self):
        face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
        drowsiness_model = load_model('drowsiness_model.h5')
        return face_cascade, drowsiness_model